# Intro

## Data Details

### Columns
'DATE, REPORT_TYPE, SOURCE, AWND, CDSD, CLDD, DSNW, DYHF, DYTS, DailyAverageDryBulbTemperature, DailyAverageStationPressure, DailyAverageWindSpeed, DailyCoolingDegreeDays, DailyDepartureFromNormalAverageTemperature, DailyHeatingDegreeDays, DailyMaximumDryBulbTemperature, DailyMinimumDryBulbTemperature, DailyPeakWindDirection, DailyPeakWindSpeed, DailyPrecipitation, DailySnowDepth, DailySnowfall, DailySustainedWindDirection, DailySustainedWindSpeed, DailyWeather, HDSD, HTDD, HourlyAltimeterSetting, HourlyDewPointTemperature, HourlyDryBulbTemperature, HourlyPrecipitation, HourlyPresentWeatherType, HourlyPressureChange, HourlyPressureTendency, HourlyRelativeHumidity, HourlySeaLevelPressure, HourlySkyConditions, HourlyStationPressure, HourlyVisibility, HourlyWetBulbTemperature, HourlyWindDirection, HourlyWindGustSpeed, HourlyWindSpeed, MonthlyDaysWithGT001Precip, MonthlyDaysWithGT010Precip, MonthlyDaysWithGT32Temp, MonthlyDaysWithGT90Temp, MonthlyDaysWithLT0Temp, MonthlyDaysWithLT32Temp, MonthlyDepartureFromNormalAverageTemperature, MonthlyDepartureFromNormalCoolingDegreeDays, MonthlyDepartureFromNormalHeatingDegreeDays, MonthlyDepartureFromNormalMaximumTemperature, MonthlyDepartureFromNormalMinimumTemperature, MonthlyDepartureFromNormalPrecipitation, MonthlyGreatestPrecip, MonthlyGreatestPrecipDate, MonthlyMaxSeaLevelPressureValue, MonthlyMaxSeaLevelPressureValueDate, MonthlyMaxSeaLevelPressureValueTime, MonthlyMaximumTemperature, MonthlyMeanTemperature, MonthlyMinSeaLevelPressureValue, MonthlyMinSeaLevelPressureValueDate, MonthlyMinSeaLevelPressureValueTime, MonthlyMinimumTemperature, MonthlySeaLevelPressure, MonthlyStationPressure, MonthlyTotalLiquidPrecipitation, NormalsCoolingDegreeDay, NormalsHeatingDegreeDay, REM, REPORT_TYPE.1, SOURCE.1, ShortDurationEndDate005, ShortDurationEndDate010, ShortDurationEndDate015, ShortDurationEndDate020, ShortDurationEndDate030, ShortDurationEndDate045, ShortDurationEndDate060, ShortDurationEndDate080, ShortDurationEndDate100, ShortDurationEndDate120, ShortDurationEndDate150, ShortDurationEndDate180, ShortDurationPrecipitationValue005, ShortDurationPrecipitationValue010, ShortDurationPrecipitationValue015, ShortDurationPrecipitationValue020, ShortDurationPrecipitationValue030, ShortDurationPrecipitationValue045, ShortDurationPrecipitationValue060, ShortDurationPrecipitationValue080, ShortDurationPrecipitationValue100, ShortDurationPrecipitationValue120, ShortDurationPrecipitationValue150, ShortDurationPrecipitationValue180, Sunrise, Sunset, MonthlyInd'

## Analysis Code

In [378]:
import pandas as pd
import numpy as np
from plotnine import *
import re
import plotly.express as px
import datetime as dt
import plotly.io as pio
pio.renderers.default = "notebook_connected"

pd.set_option('display.max_columns', None)


In [241]:
def DropNaCols(df):
    '''given a dataframe, drop all the columns with nothing but NaN'''
    naColList = []
    for ele in df.columns:
        uniqueVals = list(df[ele].unique())
        if len(uniqueVals) == 1:
            naColList.append(ele)
    return(df.drop(columns=naColList))

In [306]:
def ReadingType(df):
    '''indicates whether the given row provides a monthly, daily, or hourly reading'''
    
    #get the different column types into separate lists
    monthlyCols = [ele for ele in list(df.columns) if 'month' in ele.lower()]
    dailyCols = [ele for ele in list(df.columns) if 'dai' in ele.lower()]
    hourlyCols = [ele for ele in list(df.columns) if 'hour' in ele.lower()]

    #create columns indicating whether a row contains monthly, daily, or hourly readings
    boolMask_monthly = ~df[monthlyCols].isna()
    df['monthlyInd'] = boolMask_monthly.sum(axis=1)

    boolMask_daily = ~df[dailyCols].isna()
    df['dailyInd'] = boolMask_daily.sum(axis=1)

    boolMask_hourly = ~df[hourlyCols].isna()
    df['hourlyInd'] = boolMask_hourly.sum(axis=1)

    #if a row has more than one non-NaN value in a monthly column, the reading is monthly, else if more than one non-NaN value in a daily column, the reading is daily, else if more than one non-Nan value in an hourly column, the reading is hourly else indicate no valid readings for the row
    df['ReadingType'] = ['monthly' if df.iloc[ele]['monthlyInd'] > 0 else 'daily' if df.iloc[ele]['dailyInd'] > 0 else 'hourly' if df.iloc[ele]['hourlyInd'] > 0 else 'noValidReading' for ele in range(df.shape[0])]

    return df

In [243]:
def SplitDataframes(df):
    '''given a dataframe of labeled monthly, daily, and hourly readings, split the dataframe by those labels into component dataframes and load those to a dictionary labeled according to reading type'''
    dfDict = {}
    for ele in df['ReadingType'].unique():
        dfDict[ele] = DropNaCols(df[df['ReadingType'] == ele]).reset_index(drop=True)
    return dfDict

In [404]:
def CleanWeatherDF(df):
    '''given a dataframe of local weather data, do some cosmetic cleaning including...
    1. remove uppercase from column names and add underscores between words
    2. convert date column to datetime'''

    #1. remove uppercase from column names and add underscores between words
    new_column_names = []
    for ele in list(df.columns):
        if ele.isupper():
            new_column_names.append(ele.lower())
        else:
            new_column_names.append(re.sub('(?<!^)(?=[A-Z])', '_',ele).lower())
    df.columns = new_column_names

    # convert date column to datetime
    df['date'] = pd.to_datetime(df['date'])

    return df

In [405]:
#read in data
df = CleanWeatherDF(pd.read_csv('3063831.csv'))

#clean up column names
new_cols = []

#add reading type column
df = ReadingType(df)

#clean up data
df = DropNaCols(df)

#next, put dataframe into separate dataframes depending on the type of reading
dfDict = SplitDataframes(df)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (21,27,28,29,30,31,36,37,38,42,43,49,50,52,54,57,59,60,61,62,64,115,118,120) have mixed types.Specify dtype option on import or set low_memory=False.



In [308]:
#get all the unique values in group of columns
uniqueVal = []
for ele in dfDict['hourly'][hourlyCols].columns:
    vals = list(df[ele].unique())
    for i in vals:
        if i not in uniqueVals:
            uniqueVal.append(i)

uniqueVal

['30.11',
 '30.13',
 '30.16',
 '30.18',
 '30.19',
 '30.23',
 '30.28',
 '30.31',
 '30.32',
 '30.33',
 '30.34',
 '30.35',
 '30.30',
 '30.29',
 nan,
 '30.27',
 '30.26',
 '30.24',
 '30.20',
 '30.17',
 '30.15',
 '30.14',
 '30.12',
 '30.08',
 '30.09',
 '30.07',
 '30.05',
 '30.06',
 '30.03',
 '29.98',
 '29.96',
 '29.95',
 '29.94',
 '29.92',
 '29.90',
 '29.89',
 '29.88',
 '29.87',
 '29.85',
 '29.83',
 '29.82',
 '29.81',
 '29.80',
 '29.78',
 '29.79',
 '29.86',
 '29.97',
 '29.99',
 '30.01',
 '30.02',
 '30.04',
 '30.10',
 '29.93',
 '29.84',
 '29.91',
 '30.21',
 '30.25',
 '30.36',
 '30.38',
 '30.40',
 '30.42',
 '30.44',
 '30.45',
 '30.46',
 '30.47',
 '30.48',
 '30.49',
 '30.50',
 '30.52',
 '30.54',
 '30.56',
 '30.55',
 '30.51',
 '30.43',
 '30.39',
 '30.41',
 '30.37',
 '30.22',
 '30.00',
 '29.77',
 '29.74',
 '29.69',
 '29.67',
 '29.68',
 '29.70',
 '29.65',
 '29.64',
 '29.63',
 '29.62',
 '29.66',
 '29.71',
 '29.75',
 '29.76',
 '29.73',
 '29.72',
 '30.53',
 '30.58',
 '30.59',
 '30.61',
 '30.63',
 '30

In [354]:
numerical_columns = ['HourlyAltimeterSetting','HourlyDewPointTemperature','HourlyDryBulbTemperature','HourlyPrecipitation','HourlyPressureChange','HourlyPressureTendency','HourlyRelativeHumidity','HourlySeaLevelPressure','HourlyStationPressure','HourlyVisibility','HourlyWetBulbTemperature','HourlyWindDirection','HourlyWindGustSpeed','HourlyWindSpeed']

In [356]:
for ele in numerical_columns:
    dfDict['hourly'][ele] = dfDict['hourly'][ele].replace('[A-Za-z*]','',regex=True).replace('',0).astype(float).fillna(0)

In [381]:
dfDict['hourly']['DATE']

0        2019-01-01T00:23:00
1        2019-01-01T00:53:00
2        2019-01-01T01:53:00
3        2019-01-01T02:19:00
4        2019-01-01T02:53:00
                ...         
42074    2022-08-22T21:53:00
42075    2022-08-22T22:53:00
42076    2022-08-22T23:53:00
42077    2022-08-23T00:53:00
42078    2022-08-23T01:53:00
Name: DATE, Length: 42079, dtype: object

In [380]:
dfDict['hourly'].groupby(dfDict['hourly']['DATE'].dt.month())['HourlyPreciptation']

AttributeError: Can only use .dt accessor with datetimelike values